# LSTM. 5 featerow - mag, x, y, depth, diff_days

In [47]:
import numpy as np
import tensorflow as tf
import pandas as pd
SEED = 1337
tf.random.set_seed(SEED)
np.random.seed(SEED)

In [48]:
ds_train = tf.data.Dataset.load("../data/MEM_ds_train/")
ds_val = tf.data.Dataset.load("../data/MEM_ds_val/")
ds_test = tf.data.Dataset.load("../data/MEM_ds_test/")

2024-01-28 05:48:48.118900: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-28 05:48:48.648841: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-28 05:48:48.648914: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-28 05:48:48.662702: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-28 05:48:48.662778: I tensorflow/compile

In [49]:
ds_train.cardinality().numpy(), ds_val.cardinality().numpy(), ds_test.cardinality().numpy()

(3216086, 465519, 100158)

In [50]:
BATCH_SIZE = 512
BLOCK_SIZE = 64

In [51]:
def transform_element(features, context, labels):
    return features, (labels[:,:-1], labels[:,-1])

In [52]:
ds_train = ds_train.map(transform_element)
ds_val = ds_val.map(transform_element)
ds_test = ds_test.map(transform_element)

In [53]:
for x, y in ds_train.batch(BATCH_SIZE).take(1):
    print(x.shape)
    print(y[0].shape)
    print(y[1].shape)

(512, 64, 8)
(512, 64, 4)
(512, 64)


In [54]:
ds_train = ds_train.batch(BATCH_SIZE).shuffle(2000, seed=SEED).prefetch(tf.data.experimental.AUTOTUNE)
ds_val = ds_val.batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)

In [55]:
class MyModel(tf.keras.Model):
    def __init__(self, n_embed):
        super(MyModel, self).__init__()
        self.embed_plate = tf.keras.layers.Embedding(100, n_embed // 8)
        self.embed_dd = tf.keras.layers.Embedding(100, n_embed // 8)
        self.lstm_embed = tf.keras.layers.LSTM(n_embed // 8 * 6, return_sequences=True)
        self.lstm1 = tf.keras.layers.LSTM(n_embed, return_sequences=True)
        self.lstm2 = tf.keras.layers.LSTM(n_embed, return_sequences=True)
        self.dense1 = tf.keras.layers.Dense(n_embed, activation='relu')
        self.out1 = tf.keras.layers.Dense(4, activation='sigmoid')
        self.out2 = tf.keras.layers.Dense(17, activation='softmax')
        self.conc = tf.keras.layers.Concatenate(axis=-1)

    def call(self, x):
        cont, plate, dd = x[:,:,:-2], x[:,:,-2], x[:,:,-1] 
        x1 = self.lstm_embed(cont)
        x2 = self.embed_plate(plate)
        x3 = self.embed_dd(dd)
        x = self.conc([x1, x2, x3])
        x = self.lstm1(x)
        x = self.lstm2(x)
        x = self.dense1(x)
        out1 = self.out1(x)
        out2 = self.out2(x)
        return [out1, out2]

In [56]:
model = MyModel(128)

In [57]:
model.build(input_shape=(None, BLOCK_SIZE, 8))
model.count_params()

325141

In [58]:
LAMBDA_1 = 2
LAMBDA_2 = 1
loss_weights = {'output_1': LAMBDA_1, 'output_2': LAMBDA_2}

In [59]:
model.compile(optimizer='adam',
            loss={'output_1': 'mse', 'output_2': tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)},
            loss_weights=loss_weights,
            metrics={'output_1': 'mse', 'output_2': 'accuracy'})

In [60]:
history = model.fit(ds_train, epochs=30, validation_data=ds_val)

Epoch 1/30


2024-01-28 05:49:15.295477: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 364 of 2000
2024-01-28 05:49:35.297816: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 1086 of 2000
2024-01-28 05:49:55.298179: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 1635 of 2000
2024-01-28 05:50:04.939784: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:452] Shuffle buffer filled.
2024-01-28 05:50:09.702583: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8904
Could not load symbol cublasGetSmCountTarget from libcublas.so.11. Error: /home/majkel/miniconda3/lib/libcublas.so.11: undefined symbol: cublasGetSmCountTarget
2024-01-28 05:50:12.076320: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f44b8003

2356/6282 [==========>...................] - ETA: 10:12 - loss: 0.3474 - output_1_loss: 0.0495 - output_2_loss: 0.2485 - output_1_mse: 0.0495 - output_2_accuracy: 0.9431

2024-01-28 05:56:21.764320: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:809] failed to alloc 4294967296 bytes on host: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2024-01-28 05:56:21.764431: W ./tensorflow/compiler/xla/stream_executor/device_host_allocator.h:52] could not allocate pinned host memory of size: 4294967296
2024-01-28 05:56:21.765127: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:809] failed to alloc 3865470464 bytes on host: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2024-01-28 05:56:21.765181: W ./tensorflow/compiler/xla/stream_executor/device_host_allocator.h:52] could not allocate pinned host memory of size: 3865470464
2024-01-28 05:56:21.765488: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:809] failed to alloc 3478923264 bytes on host: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2024-01-28 05:56:21.765529: W ./tensorflow/compiler/xla/stream_executor/device_host_allocator.h:52] could not allocate pinned host memory of size: 347892

6282/6282 [==============================] - 1197s 179ms/step - loss: 0.2954 - output_1_loss: 0.0452 - output_2_loss: 0.2050 - output_1_mse: 0.0452 - output_2_accuracy: 0.9482 - val_loss: 0.2173 - val_output_1_loss: 0.0492 - val_output_2_loss: 0.1189 - val_output_1_mse: 0.0492 - val_output_2_accuracy: 0.9666
Epoch 2/30


...

6282/6282 [==============================] - 1030s 131ms/step - loss: 0.2255 - output_1_loss: 0.0260 - output_2_loss: 0.1735 - output_1_mse: 0.0260 - output_2_accuracy: 0.9487 - val_loss: 0.1812 - val_output_1_loss: 0.0340 - val_output_2_loss: 0.1131 - val_output_1_mse: 0.0340 - val_output_2_accuracy: 0.9668
Epoch 29/30


2024-01-28 16:07:55.457268: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 90 of 2000
2024-01-28 16:08:05.562600: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 189 of 2000
2024-01-28 16:08:25.457717: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 354 of 2000
2024-01-28 16:08:45.489740: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 527 of 2000
2024-01-28 16:09:05.490695: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 708 of 2000
2024-01-28 16:09:25.484991: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a w

6282/6282 [==============================] - 1029s 131ms/step - loss: 0.2251 - output_1_loss: 0.0259 - output_2_loss: 0.1734 - output_1_mse: 0.0259 - output_2_accuracy: 0.9487 - val_loss: 0.1814 - val_output_1_loss: 0.0344 - val_output_2_loss: 0.1127 - val_output_1_mse: 0.0344 - val_output_2_accuracy: 0.9668
Epoch 30/30


2024-01-28 16:25:04.410062: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 92 of 2000
2024-01-28 16:25:14.404993: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 189 of 2000
2024-01-28 16:25:34.428447: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 348 of 2000
2024-01-28 16:25:54.393827: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 530 of 2000
2024-01-28 16:26:04.481223: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 621 of 2000
2024-01-28 16:26:24.408497: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a w

6282/6282 [==============================] - 966s 121ms/step - loss: 0.2249 - output_1_loss: 0.0258 - output_2_loss: 0.1733 - output_1_mse: 0.0258 - output_2_accuracy: 0.9487 - val_loss: 0.1803 - val_output_1_loss: 0.0338 - val_output_2_loss: 0.1128 - val_output_1_mse: 0.0338 - val_output_2_accuracy: 0.9667


In [61]:
model.evaluate(ds_test)

196/196 [==============================] - 23s 113ms/step - loss: 0.1933 - output_1_loss: 0.0386 - output_2_loss: 0.1160 - output_1_mse: 0.0386 - output_2_accuracy: 0.9658


[0.19325430691242218,
 0.038622625172138214,
 0.11600910127162933,
 0.038622625172138214,
 0.96584552526474]

In [62]:
model.save("../models/MEM_model_v1.keras")

In [63]:
# save history
history = pd.DataFrame.from_dict(history.history)
history.to_csv("../models/MEM_history_model_v1.csv", index=False)